In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
import cv2
import os
import time

# Initialize Firebase Admin SDK with service account credentials
cred = credentials.Certificate("tubescc2023-firebase-adminsdk-a4hvb-b62ff9d985.json")
firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://tubescc2023.appspot.com'
})

# Define function to upload image to Firebase Storage
def upload_to_firebase(image_name):
    # Upload image to Firebase Storage
    bucket = storage.bucket()
    blob = bucket.blob(image_name)
    blob.upload_from_filename(image_name)

    # Delete local image file after uploading
    os.remove(image_name)

    print(f"Image uploaded to Firebase Storage: {blob.public_url}")

# Define GStreamer pipeline function
def gstreamer_pipeline(
    capture_width=1920,
    capture_height=1080,
    display_width=960,
    display_height=540,
    framerate=30,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink drop=True"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

def capture_image_and_upload():
    # Define image name and GStreamer pipeline
    image_name = 'image.jpg'
    pipeline = gstreamer_pipeline()

    # Create GStreamer capture object
    cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
    if not cap.isOpened():
        print("Failed to open camera.")
        return

    # Capture image
    ret, frame = cap.read()
    if ret:
        # Save image
        cv2.imwrite(image_name, frame)
        print("Image captured successfully.")

        # Upload image to Firebase
        upload_to_firebase(image_name)
    else:
        print("Failed to capture image.")

    # Release capture object
    cap.release()

# Call the function to capture image and upload to Firebase
capture_image_and_upload()
